In [2]:
import praw
import json
import time
import pandas as pd
import sys
import numpy as np

In [3]:
json_file = open('Reddit_creds.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [4]:
reddit = praw.Reddit(user_agent=json_data['user_agent'],
                     client_id=json_data['client_id'], client_secret=json_data['client_secret'])

In [5]:
yang_sub = reddit.subreddit('YangForPresidentHQ')

In [6]:
pete_sub = reddit.subreddit('Pete_Buttigieg')

In [7]:
sanders_sub = reddit.subreddit('SandersForPresident/')

In [8]:
harris_sub = reddit.subreddit('Kamala')

In [9]:
biden_sub = reddit.subreddit('JoeBiden')

In [10]:
subs = [yang_sub,pete_sub,sanders_sub,harris_sub,biden_sub]

In [11]:
users = []

In [12]:
start = time.time()
for sub in subs:
    for comment in sub.comments(limit=1000):
        users.append(comment.author)
        
print(len(users))
users = list(set(users))
print(len(users))
end = time.time()
print(end-start)

4678
1694
34.113534927368164


In [13]:
nums = np.random.randint(1,1702,100)

In [14]:
short_list = []
for x in nums:
    short_list.append(users[x])

In [16]:
start = time.time()
text = []
author = []
subreddit = []
postid = []
counter = 0
for user in short_list:
    if counter % 10 == 0:
        prog = time.time()
        print(counter)
        print((prog-start)/60)
    try:  #handle cases where redditor is deleted or something
        for comment in user.comments.new(limit=1000):
            author.append(comment.author)
            postid.append(comment.id)
            subreddit.append(comment.subreddit.display_name)
    except:
        pass
    counter += 1
end = time.time()
print(end-start)

0
2.797444661458333e-06
10
0.8571161190668742
20
1.4855206648508708
30
1.726123849550883
40
2.5624594648679095
50
3.426113951206207
60
4.1121755838394165
70
4.78797268072764
80
5.3164897163709
90
6.126079400380452
409.5078480243683


In [17]:
df = pd.DataFrame([author,subreddit,postid],\
                        index=['Author','Subreddit','Post ID']).T

In [24]:
df['Subreddit'].value_counts().head(25)

politics               6338
YangForPresidentHQ     4142
Pete_Buttigieg         3211
SandersForPresident    2431
AskReddit              1697
neoliberal             1222
worldnews              1076
MMA                    1036
stupidpol               664
Braves                  618
news                    613
fo76                    592
MaddenUltimateTeam      496
WayOfTheBern            455
Kamala                  448
movies                  442
pics                    412
todayilearned           404
freefolk                383
unpopularopinion        350
langrisser              343
Market76                340
funny                   328
Catholicism             328
Cricket                 290
Name: Subreddit, dtype: int64

In [25]:
subs = list(df['Subreddit'].value_counts().head(25).index)

In [26]:
count = 0
outer_dict = {}
for user in short_list:
    count += 1
    dict_sub = {}
    for comment in user.comments.new(limit=1000):
        if comment.subreddit.display_name in subs:
            if dict_sub.get(comment.subreddit.display_name,0) == 0:
                dict_sub[comment.subreddit.display_name] = 1
            else:
                dict_sub[comment.subreddit.display_name] += 1
    outer_dict[user] = dict_sub
    print(count)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [138]:
df = pd.DataFrame(outer_dict).T

In [139]:
def yang_gang(num):
    num = int(num)
    if num >= 10:
        return 1
    elif num == 0:
        return 0
    else:
        return 'bad'

In [140]:
df.fillna(0,inplace=True)

In [141]:
df['target']=df['YangForPresidentHQ'].apply(yang_gang)

In [142]:
df = df[(df['target']==0)|(df['target']==1)]

In [143]:
df.shape

(77, 26)

In [144]:
df.drop('YangForPresidentHQ',axis=1,inplace=True)

In [145]:
df.dropna(inplace=True)

In [146]:
from sklearn.linear_model import LogisticRegression

In [147]:
lr = LogisticRegression()

In [148]:
X = df.drop('target',axis=1)
y = df['target']

In [149]:
X = X.div(X.sum(axis=1), axis=0)

In [154]:
df2 = pd.concat([X,y],axis=1)

In [157]:
df2.dropna(inplace=True)

In [158]:
X2 = df2.drop('target',axis=1)
y2 = df2['target']

In [159]:
y2=y2.astype('int')

In [160]:
lr.fit(X2,y2)

/Users/yichenh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [161]:
lr.coef_

array([[ 0.5276791 , -0.17351823, -0.18023074, -0.09536968, -0.60877982,
         0.48179174,  0.6176701 , -0.05265871, -0.77576514, -0.39013073,
        -0.19084515, -0.0915212 , -0.09105369,  0.36652782,  0.        ,
        -0.08788976, -0.26280799, -0.16281394,  0.48864451,  0.28751587,
        -0.13765346, -0.2137676 , -0.21781012, -0.33115996]])

In [162]:
type(list(X2.columns))

list

In [163]:
pd.DataFrame(lr.coef_.T,X2.columns)

0
AskReddit            0.527679
Braves              -0.173518
Catholicism         -0.180231
Cricket             -0.095370
Kamala              -0.608780
MMA                  0.481792
MaddenUltimateTeam   0.617670
Market76            -0.052659
Pete_Buttigieg      -0.775765
SandersForPresident -0.390131
WayOfTheBern        -0.190845
fo76                -0.091521
freefolk            -0.091054
funny                0.366528
langrisser           0.000000
movies              -0.087890
neoliberal          -0.262808
news                -0.162814
pics                 0.488645
politics             0.287516
stupidpol           -0.137653
todayilearned       -0.213768
unpopularopinion    -0.217810
worldnews           -0.331160

In [167]:
df2[['politics','target']]

politics target
mrcorndogman33        0.086207      0
Pintofyourbest        0.283168      0
watercolorwildflower  0.111111      0
SecretMuslim08        0.250000      0
joebleaux             0.000000      0
fraudymcfraudster     0.553488      0
exdelicto             0.314815      0
SAKabir               0.081218      0
RxVote                0.609787      0
nevertoolate1983      0.198473      1
nykfeng               1.000000      1
Rottanathyst          0.058824      0
_AllWittyNamesTaken_  0.356725      0
JMoFilm               0.006711      0
isthisgood80          0.500000      0
squarehedge           0.447368      0
Mystic789             0.000000      0
crimpyantennae        0.010000      0
MarkJanusIsAScab      0.329949      0
DoggieSchoolBus2      0.116279      0
Milkk_Man             0.000000      1
SaltedAvocadosMhh     0.000000      0
Epic_XC               0.704545      0
Srsly_dang            0.037594      0
rabbidbunnyz22        0.303630      0
lysergic5253          0.111650      0
st1r                  0.400000      0
CordageMonger         0.000000      0
Euphoric_Translator   0.005917      0
ThoughtsOnGovernment  0.000000      0
...                        ...    ...
IamKyleBizzle         0.000000      1
MonkeysLikeApples     0.000000      0
TheRealRamanji        0.000000      0
Bizzlebomb            0.002016      1
difazx4               0.819277      0
Hodgi22               0.002755      1
Applyingto-throwaway  0.000000      0
deathhandmachiavelli  0.222222      0
LV_Mises              0.047431      0
PepitaChacha          0.008547      0
Pete4Me               0.000000      0
ChickerWings          0.381766      0
LDCrow                0.000000      0
jcwinny               0.166667      0
hoksze                0.000000      0
Theandric             0.011594      0
MusicNutt             0.063291      0
zombiefatcher         0.000000      0
albiorix_             0.665217      0
WebAPI                0.482759      1
ballisnotlife22       0.000000      1
LifeBasedDiet         0.912281      1
HowWinterKills        0.000000      0
papercutdicked        0.871795      1
backtoreality0101     0.898571      0
iwantthetopbunk       0.000000      1
dustyjeff             0.040000      0
kamalanyc             0.035714      0
MetaFlight            0.000000      0
effRPaul              0.009208      0

[71 rows x 2 columns]